In [1]:
%pip install --upgrade -q pip setuptools wheel
%pip install -q Pylance PyObjC pythainlp openai gtts matplotlib playsound icrawler fastapi gradio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import matplotlib as mpl
import os
import matplotlib.font_manager
if os.path.exists('./Sarabun-Regular.ttf') == False:
    os.system('wget -q https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf')

mpl.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
mpl.rc('font', family='Sarabun')

In [39]:
import logging
logging.disable(logging.CRITICAL)

In [2]:
import os
import shutil
import random
import matplotlib.pyplot as plt

from PIL import Image
from gtts import gTTS
from openai import OpenAI
from pythainlp.tokenize import word_tokenize
from icrawler.builtin import GoogleImageCrawler

def create_opentyphoon_client():
    API_KEY='sk-psIjnhS63MFqDJzmrCnY6iLPr3iZ6wAXP57La0dvUcNoE9dc'
    return OpenAI(
        api_key=API_KEY,
        base_url="https://api.opentyphoon.ai/v1",
    )

def get_random_file_path(directory):
    all_items = os.listdir(directory)
    files = [f for f in all_items if os.path.isfile(os.path.join(directory, f))]
    if not files:
        return None
    random_file = random.choice(files)
    return os.path.join(directory, random_file)

def cue_hierachy(state, target_word, client):
    semantic_cue = f"""คุณเป็นนักบำบัดการพูดที่กำลังช่วยผู้ป่วยที่มีภาวะเสียการพูดฝึกฝนทักษะการพูดและการสื่อสาร จากโรค Aphasia
    ผู้ป่วยอาจมีปัญหาในการหาคำและการสร้างประโยค คุณมีหน้าที่ช่วยผู้ป่วยให้สามารถคิดคำที่คุณใบ้ออก <โดยห้ามบอกถึงโดยตรง>
    การใบ้คนไข้คุณต้องอธิบายประเภทของสิ่งที่จะใบ้ เช่น
    ตัวอย่างการใบ้คุณยประเภทสำหรับคำว่า "แอปเปิ้ล"
    มันเป็นผลไม้
    ตัวอย่างการใบ้คุณยประเภทสำหรับคำว่า "รถยนต์"
    มันคือยานพาหนะ
    """

    sentence_completion_cue = f"""คุณเป็นนักบำบัดการพูดที่กำลังช่วยผู้ป่วยที่มีภาวะเสียการพูดฝึกฝนทักษะการพูดและการสื่อสาร จากโรค Aphasia
    ผู้ป่วยอาจมีปัญหาในการหาคำและการสร้างประโยค คุณมีหน้าที่ช่วยผู้ป่วยให้สามารถคิดคำที่คุณใบ้ออก <โดยห้ามบอกถึงโดยตรง>
    การใบ้คนไข้คุณต้องสร้างประโยคที่กำลังใช้หรือมีปฏิสัมพันธ์ของสิ่งที่จะใบ้ เช่น
    ตัวอย่างการใบ้คำว่า "แอปเปิ้ล"
    ผมกำลังเดินทางไปตลาดเผื่อซื้อผลไม้สีแดงสดมาทำขนมพาย
    """

    phonemic_cue = f"""คุณเป็นนักบำบัดการพูดที่กำลังช่วยผู้ป่วยที่มีภาวะเสียการพูดฝึกฝนทักษะการพูดและการสื่อสาร จากโรค Aphasia
    ผู้ป่วยอาจมีปัญหาในการหาคำและการสร้างประโยค คุณมีหน้าที่ช่วยผู้ป่วยให้สามารถคิดคำที่คุณใบ้ออก <โดยห้ามบอกถึงโดยตรง>
    การใบ้คนไข้คุณต้องอธิบายลักษณะของเสียงพยัญชนะแรกของสิ่งที่จะใบ้ของคำๆนั้น เช่น
    ตัวอย่างการใบ้คำว่า "แอปเปิ้ล"
    เสียง ออ
    ตัวอย่างการใบ้คำว่า "กล้วย"
    เสียง กอ
    ตัวอย่างการใบ้คำว่า "คน"
    เสียง คอ
    """

    functional_description_cue = f"""คุณเป็นนักบำบัดการพูดที่กำลังช่วยผู้ป่วยที่มีภาวะเสียการพูดฝึกฝนทักษะการพูดและการสื่อสาร จากโรค Aphasia
    ผู้ป่วยอาจมีปัญหาในการหาคำและการสร้างประโยค คุณมีหน้าที่ช่วยผู้ป่วยให้สามารถคิดคำที่คุณใบ้ออก <โดยห้ามบอกถึงโดยตรง>
    การใบ้คนไข้คุณต้องอธิบายลักษณะของสิ่งที่จะใบ้ เช่น
    ตัวอย่างการใบ้คุณลักษณะสำหรับคำว่า "แอปเปิ้ล"
    มันเป็นผลไม้ที่เอาไว้กิน มีสีแดงหรือสีเขียว มีผิวที่กลมและเรียบ มีขายตามห้างหรือร้านขายผลไม้
    """
    
    if state == 0:
        mode = semantic_cue
    elif state == 1:
        mode = sentence_completion_cue
    elif state == 2:
        mode = phonemic_cue
    elif state == 4:
        mode = functional_description_cue
    else:
        return None

    stream = client.chat.completions.create(
    model="typhoon-instruct",
    messages=[
        {
            "role": "systemp",
            "content": f'{mode}',
        },
        {
            "role": "user",
            "content": f"ผมอยากพูดคำว่า {target_word}",
        }
    ],
    
    max_tokens=300,
    temperature=0,
    top_p=0.99,
    stream=True,
    )

    respond=[]
    for chunk in stream:
        if hasattr(chunk, 'choices') and len(chunk.choices) > 0:
            choice = chunk.choices[0]
            if hasattr(choice, 'delta') and hasattr(choice.delta, 'content'):
                if choice.delta.content is not None:
                    respond.append(choice.delta.content)

    return "".join(respond).replace(target_word, '')

In [7]:
client = create_opentyphoon_client()

def call_cue_hierarchy(state, target_word, client=client):
    respond=None
    audio_path=None
    img_path=None
    if state >= 0 and state <= 2:
        respond = cue_hierachy(state, target_word, client)
    elif state == 3:
        tts = gTTS(target_word, lang='th')
        tts.save('temp.mp3')
        audio_path='./temp.mp3'
    elif state == 4:
        if os.path.exists('./img'):
            shutil.rmtree('./img')
            
        google_crawler = GoogleImageCrawler(storage={'root_dir': './img'})
        google_crawler.crawl(keyword=target_word, max_num=3)
        img_path=get_random_file_path('./img')
    elif state == 5:
        tts = gTTS(target_word, lang='th')
        tts.save('temp.mp3')
        audio_path='./temp.mp3'
        plt.figure(figsize=(4, 2))
        plt.text(0.5, 0.5, target_word, fontsize=36, ha='center', va='center', transform=plt.gca().transAxes)
        plt.axis('off')
        img_path='./target_word.jpg'
        plt.savefig('./target_word.jpg')
    return_format={"state" : state}
    if respond != None:
        return_format.update({"respond" : respond,})
    if audio_path != None:
        return_format.update({"audio_path": audio_path})
    if img_path != None:
        return_format.update({"img_path": img_path})
    return return_format
        
def check_user_input(target_word, user_input):
    user_respond = word_tokenize(user_input, engine="newmm")
    if target_word in user_respond:
        return True
    else:
        return False

0.Semantic Cue : อธิบายประเภท

1.Sentence Completion Cue :สร้างประโยคที่เกี่ยวข้อง

2.Phonemic Cue : อธิบายเสียงแรก

3.Modeling Cue : เล่นเสียง

4.Functional Description Cue : อธิบายรายละเอียด

5.Printed Word Cue : เปิดคำ

In [64]:
for i in range(6):
    print(call_cue_hierarchy(i,'ทนาย'))

{'state': 0, 'respond': 'เป็นผู้ที่มีความรู้และความชำนาญในการกฎหมาย เขาจะช่วยคุณในการจัดการกับคดีความหรือข้อพิพาทต่างๆ'}
{'state': 1, 'respond': 'เมื่อคุณไปศาล คุณต้องการผู้ช่วยที่มีความรู้กฎหมายและสามารถอธิบายความหมายของกฎหมายให้คุณเข้าใจ'}
{'state': 2, 'respond': 'เสียง ทอ'}
{'state': 3, 'audio_path': './temp.mp3'}
{'state': 4, 'img_path': './img/000002.png'}
{'state': 5, 'audio_path': './temp.mp3', 'img_path': './target_word.jpg'}


In [ ]:
import gradio as gr
import random

target_word='ทนาย'
state=1
def config_gradio(user_input):
    global state
    if state <= 6:
        respond=call_cue_hierarchy(state,target_word)
        if state == 1 or state == 2:
            state=state+1
            return respond['state'], respond['respond'], None, None, check_user_input(target_word, user_input)
        elif state == 3:
            state=state+1
            return respond['state'], None, None, respond['audio_path'], check_user_input(target_word, user_input)
        elif state == 4:
            state=state+1
            return respond['state'], None, respond['img_path'], None, check_user_input(target_word, user_input)
        elif state == 5:
            state=state+1
            return respond['state'], None, respond['img_path'], respond['audio_path'], check_user_input(target_word, user_input)

outputs = [gr.Textbox(label="State", value=state-1), gr.Textbox(label="question", value=call_cue_hierarchy(0,'ทนาย')['respond']), gr.Image(label="Image question"), gr.Audio(label="Audio question"), gr.Textbox(label="Check Answer")]
    
demo = gr.Interface(fn=config_gradio,
             inputs=gr.Textbox(label="Enter your word"),
             
             outputs=outputs,
             )

demo.launch(share=True)

2024-06-30 00:05:12,766 - INFO - httpx - HTTP Request: POST https://api.opentyphoon.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-30 00:05:13,348 - INFO - httpx - HTTP Request: GET http://127.0.0.1:7880/startup-events "HTTP/1.1 200 OK"
2024-06-30 00:05:13,355 - INFO - httpx - HTTP Request: HEAD http://127.0.0.1:7880/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7880


2024-06-30 00:05:14,136 - INFO - httpx - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2024-06-30 00:05:14,187 - INFO - httpx - HTTP Request: GET https://api.gradio.app/v2/tunnel-request "HTTP/1.1 200 OK"
2024-06-30 00:05:14,384 - INFO - httpx - HTTP Request: GET https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_darwin_arm64 "HTTP/1.1 200 OK"


Running on public URL: https://255bef785616dca41a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


2024-06-30 00:05:17,042 - INFO - httpx - HTTP Request: HEAD https://255bef785616dca41a.gradio.live "HTTP/1.1 200 OK"


2024-06-30 00:07:14,054 - INFO - httpx - HTTP Request: POST https://api.opentyphoon.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-30 00:07:38,991 - INFO - httpx - HTTP Request: POST https://api.opentyphoon.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-30 00:07:53,621 - INFO - icrawler.crawler - start crawling...
2024-06-30 00:07:53,622 - INFO - icrawler.crawler - starting 1 feeder threads...
2024-06-30 00:07:53,623 - INFO - feeder - thread feeder-001 exit
2024-06-30 00:07:53,624 - INFO - icrawler.crawler - starting 1 parser threads...
2024-06-30 00:07:53,625 - INFO - icrawler.crawler - starting 1 downloader threads...
2024-06-30 00:07:54,329 - INFO - parser - parsing result page https://www.google.com/search?q=%E0%B8%97%E0%B8%99%E0%B8%B2%E0%B8%A2&ijn=0&start=0&tbs=&tbm=isch
2024-06-30 00:07:54,476 - INFO - downloader - image #1	https://www.lawgeneration.net/wp-content/uploads/2022/05/lawyer-don-3.jpg
2024-06-30 00:07:54,478 - ERROR - downloader - Exception caught when downloading